In [1]:
# Imports
import pandas as pd
# from sqlalchemy import create_engine
# import sqlalchemy
import os,sys
from pathlib import Path

In [2]:
# Settings
path_to_data = "../example_data/example_csv"

In [3]:
# All magic here!

# Create result dict
pandases = {}
# Don't forget to add check for the shema name occupied
pandases["database_schema"] = pd.DataFrame()

database_schema_dict = {}

def get_dataframes_from_csv_folder(pandases_dict , path_to_data , parent_folder = None):
    '''Recurcively read all dataframes to one dict'''
    for file in os.listdir(path_to_data):
        full_path = os.path.join( path_to_data, file)
        if os.path.isfile(full_path):
            # Do stuff with files
            filename_no_extention = file.split('.')[0]
            
            if parent_folder:
                database_schema_dict["parent_folder"] = parent_folder
                database_schema_dict["table_name"] = f"{parent_folder}_{filename_no_extention}"
                dataframe_key = f"{parent_folder}_{filename_no_extention}"
            else:
                database_schema_dict["parent_folder"] = ""
                database_schema_dict["table_name"] = filename_no_extention
                dataframe_key = filename_no_extention
            print(f"Adding: {file} to {filename_no_extention}")
            dataframe = pd.read_csv( full_path , index_col=0 )
            pandases_dict[dataframe_key] = dataframe
            database_schema_dict["colomns"] = list(dataframe.columns)
            # Create schema colomn and push data about this table there.
            series = pd.Series(database_schema_dict)
            pandases["database_schema"][database_schema_dict["table_name"]] = series
            
        elif ( os.path.isdir ( full_path ) ):
            # Do stuff with dirs
            print(f"Dir {file}")
            get_dataframes_from_csv_folder(pandases_dict , full_path , parent_folder = file )
    return pandases_dict

pandases = get_dataframes_from_csv_folder(pandases.copy() , path_to_data)

Adding: emotions_metrics.csv to emotions_metrics
Adding: emotions_data.csv to emotions_data
Adding: statement_irt_result.csv to statement_irt_result
Dir dynamics_dominants
Adding: Wild & Fork 39s (1).csv to Wild & Fork 39s (1)
Adding: Kavak.csv to Kavak
Adding: Coppel Moda.csv to Coppel Moda
Adding: Coppel Seleccion.csv to Coppel Seleccion
Dir dynamics
Adding: Wild & Fork 39s (1).csv to Wild & Fork 39s (1)
Adding: Kavak.csv to Kavak
Adding: Coppel Moda.csv to Coppel Moda
Adding: Coppel Seleccion.csv to Coppel Seleccion


In [4]:
pandases["database_schema"]

,emotions_metrics,emotions_data,statement_irt_result,dynamics_dominants_Wild & Fork 39s (1),dynamics_dominants_Kavak,dynamics_dominants_Coppel Moda,dynamics_dominants_Coppel Seleccion,dynamics_Wild & Fork 39s (1),dynamics_Kavak,dynamics_Coppel Moda,dynamics_Coppel Seleccion
parent_folder,,,,dynamics_dominants,dynamics_dominants,dynamics_dominants,dynamics_dominants,dynamics,dynamics,dynamics,dynamics
table_name,emotions_metrics,emotions_data,statement_irt_result,dynamics_dominants_Wild & Fork 39s (1),dynamics_dominants_Kavak,dynamics_dominants_Coppel Moda,dynamics_dominants_Coppel Seleccion,dynamics_Wild & Fork 39s (1),dynamics_Kavak,dynamics_Coppel Moda,dynamics_Coppel Seleccion
colomns,"[stimulus, metrics, value]","[Video, Status, Timestamp, CALM, CONFUSED, FEA...","[stimulus, message, weight, Coppel Moda.mp4, C...","[Timestamp, CALM, ANGRY, CONFUSED, SAD, SURPRI...","[Timestamp, CALM, ANGRY, CONFUSED, SAD, SURPRI...","[Timestamp, CALM, ANGRY, CONFUSED, SAD, SURPRI...","[Timestamp, CALM, ANGRY, CONFUSED, SAD, SURPRI...","[Timestamp, ANGRY, CONFUSED, SAD, SURPRISED, D...","[Timestamp, ANGRY, CONFUSED, SAD, SURPRISED, D...","[Timestamp, ANGRY, CONFUSED, SAD, SURPRISED, D...","[Timestamp, ANGRY, CONFUSED, SAD, SURPRISED, D..."


In [5]:
# Temp dict to save as row of one database data
database_schema_dict

{'parent_folder': 'dynamics',
 'table_name': 'dynamics_Coppel Seleccion',
 'colomns': ['Timestamp',
  'ANGRY',
  'CONFUSED',
  'SAD',
  'SURPRISED',
  'DISGUSTED',
  'FEAR',
  'HAPPY',
  'ENGAGEMENT']}

In [5]:
pandases.keys()

dict_keys(['database_schema'])

In [6]:
pandases['statement_irt_result']

KeyError: 'statement_irt_result'

In [9]:
!ls ..

bkup		    Dockerfile	     README.md	       test_data_source_api
data		    example_data     requirements.txt  test_tools
deploy.sh	    grafana-storage  shell_scripts
docker-compose.yml  grafana-volume   streamlit_app


In [10]:
temp_df = pd.read_csv("../example_data/csv/statement_irt_result.csv", index_col=0)
temp_df

FileNotFoundError: [Errno 2] No such file or directory: '../example_data/csv/statement_irt_result.csv'

In [5]:
s = pd.Series(database_schema_dict)
s

/tmp/ipykernel_11929/2641468747.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(database_schema_dict)


Series([], dtype: float64)

In [6]:
temp_df = pd.DataFrame()
temp_df["123"] = s

In [7]:
temp_df =  pd.DataFrame.from_dict(database_schema_dict, orient='columns')

In [8]:
temp_df

""


In [9]:
# Create engine:
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Get script path
my_path = os.getcwd()

# Get settings from .env
env_folder = f"{my_path}/../.env"
load_dotenv(env_folder)  # take environment variables from .env.

# Make database connection
os.environ["DB_HOST"] = "localhost"
database_string = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(database_string)

In [10]:
for dataframe_name in pandases.keys():
    pandases[dataframe_name].to_sql(dataframe_name, engine)

ValueError: Table 'database_schema' already exists.